In [4]:
import spikeinterface as si
import matplotlib.pyplot as plt
%matplotlib inline
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import numpy as np
import pandas as pd
import seaborn as sns

- Dynamic

In [5]:
def get_trigger_time_range(file_path):
    recording = se.read_blackrock(file_path=file_path)
    recording = recording.channel_slice(channel_ids=['98'])
    trace_mat = recording.get_traces(start_frame= 10000,
                                    end_frame= 1350 * 10000,
                                    channel_ids= ['98'])
    trace_mat = trace_mat.astype(int)
    trace_mat[trace_mat < 20000] = 0
    trace_mat[trace_mat >= 20000] = 1

    starts = []
    ends = []

    for i in range(1, len(trace_mat)):
        if trace_mat[i] == 1 and trace_mat[i-1] == 0:
            starts.append(i+1)
        elif trace_mat[i] == 0 and trace_mat[i-1] == 1:
            ends.append(i)

    df = pd.DataFrame({'start': starts, 'end': ends})
    df = df[2:-2]
    df['end'] = df['start'] + 5000
    date = file_path.split("/")[8]
    date = date.split("_")[1]
    df['date'] = date
    return df


In [6]:
import os
recording_dir = os.listdir("/media/ubuntu/sda/data/mouse6/ns4/grating")
trigger_time = pd.DataFrame()
#recording_dir.remove("mouse6_102122_natural_image_001.ns4")

for recording in recording_dir:
    temp = get_trigger_time_range(f'/media/ubuntu/sda/data/mouse6/ns4/grating/{recording}')
    trigger_time = pd.concat((trigger_time, temp), axis = 0)

In [7]:
for date in trigger_time['date'].unique():
    temp = trigger_time[trigger_time['date'] == date]
    print(f'{date}: {len(temp)}')

042422: 180
031722: 180
072322: 180
062422: 180
042323: 180
032123: 180
022223: 180
102122: 180
052422: 180
022522: 180
112022: 180
082322: 180
092422: 180
021322: 180


###### Trigger order

In [8]:
def get_image_order(file_path):
    from scipy.io import loadmat
    mat_data = loadmat(file_path)
    mat_data = mat_data['Image_show_indices_dynamic']
    mat_data_pure = mat_data.ravel()

    row_indices = np.repeat(np.arange(mat_data.shape[0]), mat_data.shape[1])
    suffixes = np.array([f"_{i+1}" for i in row_indices])
    mat_data_order = np.array([f"{val}{suffix}" for val, suffix in zip(mat_data_pure, suffixes)])
    return mat_data_pure, mat_data_order

In [9]:
import os
trigger_dir = os.listdir("/media/ubuntu/sda/data/trigger/mouse6/")

In [10]:
date_order = ['021322', '022223', '022522', '031722', '032123', '042323', '042422', '052422', '062422', '072322', '082322', '092422', '102122', '112022']

In [11]:
image_order_dict = {}
for date in date_order:
    image_order_dict[date] = {}
    trigger = [s for s in trigger_dir if date in s][0]
    mat_dir = os.listdir(f"/media/ubuntu/sda/data/trigger/mouse6/{trigger}/")
    mat = [s for s in mat_dir if 'Dynamic' in s][0]
    image_order_dict[date]['pure'], image_order_dict[date]['order'] = get_image_order(f"/media/ubuntu/sda/data/trigger/mouse6/{trigger}/{mat}")

In [12]:
trigger_time['image'] = None
trigger_time['order'] = None
for date in trigger_time['date'].unique():
    trigger_time.loc[trigger_time['date'] == date, 'image'] = image_order_dict[date]['pure']
    trigger_time.loc[trigger_time['date'] == date, 'order'] = image_order_dict[date]['order']

In [13]:
trigger_time.to_csv('/media/ubuntu/sda/data/mouse6/output_grating/01_get_trigger/trigger_time_dynamic.tsv', sep = '\t')


- Stat

In [14]:
def get_trigger_time_range(file_path):
    recording = se.read_blackrock(file_path=file_path)
    recording = recording.channel_slice(channel_ids=['98'])
    trace_mat = recording.get_traces(start_frame= 1350 * 10000,
                                    end_frame= recording.get_duration() * 10000,
                                    channel_ids= ['98'])
    trace_mat = trace_mat.astype(int)
    trace_mat[trace_mat < 20000] = 0
    trace_mat[trace_mat >= 20000] = 1

    starts = []
    ends = []

    for i in range(1, len(trace_mat)):
        if trace_mat[i] == 1 and trace_mat[i-1] == 0:
            starts.append(i+1)
        elif trace_mat[i] == 0 and trace_mat[i-1] == 1:
            ends.append(i)

    df = pd.DataFrame({'start': starts, 'end': ends})
    df = df[2:-2]
    df['end'] = df['start'] + 5000
    date = file_path.split("/")[8]
    date = date.split("_")[1]
    df['date'] = date
    return df

In [15]:
import os
recording_dir = os.listdir("/media/ubuntu/sda/data/mouse6/ns4/grating")
trigger_time = pd.DataFrame()
#recording_dir.remove("mouse6_102122_natural_image_001.ns4")

for recording in recording_dir:
    temp = get_trigger_time_range(f'/media/ubuntu/sda/data/mouse6/ns4/grating/{recording}')
    trigger_time = pd.concat((trigger_time, temp), axis = 0)

In [16]:
for date in trigger_time['date'].unique():
    temp = trigger_time[trigger_time['date'] == date]
    print(f'{date}: {len(temp)}')

042422: 270
031722: 270
072322: 270
062422: 270
042323: 270
032123: 270
022223: 270
102122: 270
052422: 270
022522: 270
112022: 270
082322: 270
092422: 270
021322: 270


In [17]:
def get_image_order(file_path):
    from scipy.io import loadmat
    mat_data = loadmat(file_path)
    mat_data = mat_data['Image_show_indices_static']
    mat_data_pure = mat_data.ravel()

    row_indices = np.repeat(np.arange(mat_data.shape[0]), mat_data.shape[1])
    suffixes = np.array([f"_{i+1}" for i in row_indices])
    mat_data_order = np.array([f"{val}{suffix}" for val, suffix in zip(mat_data_pure, suffixes)])
    return mat_data_pure, mat_data_order

In [18]:
image_order_dict = {}
for date in date_order:
    image_order_dict[date] = {}
    trigger = [s for s in trigger_dir if date in s][0]
    mat_dir = os.listdir(f"/media/ubuntu/sda/data/trigger/mouse6/{trigger}/")
    mat = [s for s in mat_dir if 'Static' in s][0]
    image_order_dict[date]['pure'], image_order_dict[date]['order'] = get_image_order(f"/media/ubuntu/sda/data/trigger/mouse6/{trigger}/{mat}")

trigger_time['image'] = None
trigger_time['order'] = None
for date in trigger_time['date'].unique():
    trigger_time.loc[trigger_time['date'] == date, 'image'] = image_order_dict[date]['pure']
    trigger_time.loc[trigger_time['date'] == date, 'order'] = image_order_dict[date]['order']

trigger_time.to_csv('/media/ubuntu/sda/data/mouse6/output_grating/01_get_trigger/trigger_time_static.tsv', sep = '\t')